In [ ]:
import urllib.request
import urllib.parse
import tempfile
import scrapbook
import uuid
import textwrap
import xarray as xr

In [ ]:
query_string = urllib.parse.urlencode({**args, "rangeSubset": ",".join(source_bands)})
url = f"{collection}/coverage?{query_string}"

print(f"Retrieving data from {url}")

with urllib.request.urlopen(url) as in_file:
    with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmp_file:
        # TODO: use some kind of memory file
        tmp_file.write(in_file.read())
        tmp_file.flush()
        arr = xr.open_rasterio(tmp_file.name)


In [ ]:
ds = [arr.to_dataset("band")]
# set names of dataset
ds[0] = ds[0].rename({
    i: band
    for i, band in enumerate(
        source_bands,
        start=1,
    )
})

In [ ]:
if "range_subset" not in locals():
    range_subset = bands_python_functions.keys()

In [ ]:
result = xr.Dataset()
for band_name, fun_code in bands_python_functions.items():
    if band_name not in range_subset:
        print(f"skipping {band_name}")
        continue
    print(f"Executing code: {fun_code}")
    body = textwrap.indent(fun_code, " " * 4)
    exec(f"def fun():\n{body}")
    result[band_name] = fun()
    print("got result:")
    print(result[band_name])


In [ ]:
!python3 -m pip -q install rioxarray
import rioxarray

output_filepath = f"/home/jovyan/s3/ndvi-{uuid.uuid4()}.tif"

print(f"Writing output to {output_filepath}")

result.rio.to_raster(output_filepath)

scrapbook.glue("result-file", output_filepath)